# Load functions and dask cluster

In [15]:
%load_ext autoreload
%autoreload 2
from processing_imports_and_functions import *

print(os.getcwd())

# Configure dask cluster (adjust n_workers & memory_limit as needed)
cluster = LocalCluster(
    n_workers=10,           
    threads_per_worker=1,       
    memory_limit='7GB',     # 10 workers * 7GB = 70GB max RAM
    dashboard_address=':8787'
)

client = cluster.get_client()
print(f"Dashboard: {client.dashboard_link}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/mnt/ka_gh_storage/2025_11_27_40x_persister_1HourAmpicillinTreatments


/home/ka537/miniconda3/envs/extraction/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36707 instead
  warnings.warn(


Dashboard: http://127.0.0.1:36707/status


# Segmentation mask filtering

In [2]:
# Set filtering criteria (to remove segmentation artefacts)
TRENCHES = "trenches.zarr"
MASKS = "masks_upscaled.zarr"
CONFIG = "config.json"
UPSCALE = True                # set True if masks were 2x2 upscaled during segmentation

FILTERING_CONFIG = {
    'min_area':   {'min': 0,   'max': 300, 'step': 5,    'default': 100},
    'solidity':   {'min': 0.0, 'max': 1.0, 'step': 0.01, 'default': 0.86},
    'top_limit':  {'min': 0,   'max': 50, 'step': 1,    'default': 10},
    'side_limit': {'min': 0,   'max': 10,  'step': 1,    'default': 2}
}

filtering_params = run_interactive_filtering(TRENCHES, MASKS, CONFIG, FILTERING_CONFIG, upscale=UPSCALE)

Napari running... Adjust sliders, then click 'Finish and Save Parameters'.


In [3]:
# Run filtering criteria for all mother_ids
OUTPUT_DIRECTORY = "masks_upscaled_filtered.zarr"

run_batch_filtering(input_path=MASKS, output_path=OUTPUT_DIRECTORY, filtering_params=filtering_params)

Input Shape: (60420, 321, 1, 270, 48) | Chunk Size: (76, 321, 1, 270, 48)
Using Active Dask Cluster: http://127.0.0.1:8787/status
Batch processing complete. Time taken: 26.29 minutes.


In [16]:
# Final check that filtered masks look good:
run_comparison_viewer(trenches_path=TRENCHES, masks_orig_path=MASKS, masks_filt_path=OUTPUT_DIRECTORY, config_path=CONFIG, upscale=UPSCALE)

Comparison Viewer: Opening...
Calculated contrast limits from first frame...


In [ ]:
# If filtered masks still don't look great, consider re-training the segmentation model and re-running segmentation, or implementing filters based on xy centroids of the top masks (i.e. if they jump around a lot) etc.

In [ ]:
#